In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('Event_log.txt', sep='\t', encoding='cp1251')

In [3]:
df.head(3)
# Amount, Material, supplier, user SOD money amount

,CaseID,Event end,Activity,Activity Category,Activity Class,Activity type,User,Department,Amount,Purchase type,Supplier,Supplier risk level,Supplier Size,Supplier_type,INVOICE,Cleared invoice,PO TYPE,Material,User_type
0,2519 - 236,2017-04-25 08:52:27.000,Авансовый платеж,Авансовый платеж,Авансовый платеж,Авансовый платеж,NaN,Uncategorized,144.775,Услуги,Sellers of Cycles,Low,Unknown,Финансовые учреждения,NaN,NaN,Заказ на оформление закупки,Credit for Black Touring End Caps,NaN
1,2519 - 236,2017-04-24 10:32:53.000,Требование авансового платежа,Авансовый платеж,Авансовый платеж,Авансовый платеж,NaN,Uncategorized,144.775,Услуги,Sellers of Cycles,Low,Unknown,Финансовые учреждения,NaN,NaN,Заказ на оформление закупки,Credit for Black Touring End Caps,NaN
2,2519 - 236,2017-04-25 08:59:34.000,Перерасчет авансового платежа,Перерасчет авансового платежа,Платеж (выравнивание),Перерасчет авансового платежа,NaN,Uncategorized,144.775,Услуги,Sellers of Cycles,Low,Unknown,Финансовые учреждения,NaN,NaN,Заказ на оформление закупки,Credit for Black Touring End Caps,NaN


In [5]:
#SOD violation detection programm
# all following events are in "activity" category

#incomp_events = ['Заказ на поставку создан', 'Создание Заявки', 'Требование авансового платежа-Возврат', 'Счет заведен']

incomp_event1 = 'Заказ на поставку создан'
incomp_event2 = 'Создание Заявки'
incomp_event3 = 'Требование авансового платежа-Возврат'
incomp_event4 = 'Счет заведен'

# following activities are incompatible with previous ones

incomp_with1 = ['Заказ на поставку согласован 1', 'Заказ на поставку согласован 2', 'Платеж (выравнивание)', 
                'Заказ на поставку блокирован', 'Заказ на поставку: изменен статус выпуска: ДанныеОтпр, возможны изменения',
                'Заказ на поставку: изменен статус выпуска: ДанныеСогл, измен не возможны', 
                'Заказ на поставку: изменен статус выпуска: НачКод, возможны изменения', 
                'Заказ на поставку: изменен статус выпуска: Согл, измен не возможны']

incomp_with2 = ['Заявка согласована']
incomp_with3 = ['Авансовый платеж', 'Перерасчет авансового платежа', 'Перерасчет авансового платежа-Возврат', 
                'Перерасчет по требованию авансового платежа']
incomp_with4 = ['Счет блокирован: несоответствие цены', 'Счет предварительно полностью зарегистрирован', 
                'Счет блокирован: несоответствие даты', 
                'Счет предварительно зарегистрирован']

users = df['User'].unique()
users = list(users)
# first element of users_activity_track[i] is a user himself, second element is a quntity of times user broke SOD,
# third elementis a amount of money in cases in wich user broke SOD
# and after that all activities user has done in the following order first comes the activity and second is case id in
# which that activity occured example [user_11, 0, 0, 'Счет блокирован: несоответствие цены', 2519 - 236]

any_in = lambda a, b: any(i in b for i in a)
users_activity_track = []
current_case_activ_list = []
for i in range(len(users)):
    users_activity_track.append([users[i], 0, 0])
for j in range(len(df["CaseID"])):
    current_case_activ_list = []
    user_index = users.index(df['User'][j])
    activity = df["Activity"][j]
    users_activity_track[user_index].append(activity)
    users_activity_track[user_index].append(df["CaseID"][j])
    if(activity == incomp_event1):
        for n in range(len(users_activity_track[user_index])):
            if(df["CaseID"][j] == users_activity_track[user_index][n]):
                current_case_activ_list.append(users_activity_track[user_index][n - 1])
            if(activity == users_activity_track[user_index][n] and df["CaseID"][j] == users_activity_track[user_index][n + 1]):
                if(any_in(incomp_with1, current_case_activ_list)):
                    users_activity_track[user_index][1] += 1
                    if(not np.isnan(df["Amount"][j])):
                        users_activity_track[user_index][2] += df["Amount"][j]
                    break
        
    if(activity == incomp_event2):
        for n in range(len(users_activity_track[user_index])):
            if(df["CaseID"][j] == users_activity_track[user_index][n]):
                current_case_activ_list.append(users_activity_track[user_index][n - 1])
            if(activity == users_activity_track[user_index][n] and df["CaseID"][j] == users_activity_track[user_index][n + 1]):
                if(any_in(incomp_with2, current_case_activ_list)):
                    users_activity_track[user_index][1] += 1
                    if(not np.isnan(df["Amount"][j])):
                        users_activity_track[user_index][2] += df["Amount"][j]
                    break    
    
    if(activity == incomp_event3):
        for n in range(len(users_activity_track[user_index])):
            if(df["CaseID"][j] == users_activity_track[user_index][n]):
                current_case_activ_list.append(users_activity_track[user_index][n - 1])
            if(activity == users_activity_track[user_index][n] and df["CaseID"][j] == users_activity_track[user_index][n + 1]):
                if(any_in(incomp_with3, current_case_activ_list)):
                    users_activity_track[user_index][1] += 1
                    if(not np.isnan(df["Amount"][j])):
                        users_activity_track[user_index][2] += df["Amount"][j]
                    break 

    if(activity == incomp_event4):
        for n in range(len(users_activity_track[user_index])):
            if(df["CaseID"][j] == users_activity_track[user_index][n]):
                current_case_activ_list.append(users_activity_track[user_index][n - 1])
            if(activity == users_activity_track[user_index][n] and df["CaseID"][j] == users_activity_track[user_index][n + 1]):
                if(any_in(incomp_with4, current_case_activ_list)):
                    users_activity_track[user_index][1] += 1
                    if(not np.isnan(df["Amount"][j])):
                        users_activity_track[user_index][2] += df["Amount"][j]
                    break
    print("j:", j)
for k in range(len(users_activity_track)):
    print(users_activity_track[k][0], users_activity_track[k][1])

j: 0
j: 1
j: 2
j: 3
j: 4
j: 5
j: 6
j: 7
j: 8
j: 9
j: 10
j: 11
j: 12
j: 13
j: 14
j: 15
j: 16
j: 17
j: 18
j: 19
j: 20
j: 21
j: 22
j: 23
j: 24
j: 25
j: 26
j: 27
j: 28
j: 29
j: 30
j: 31
j: 32
j: 33
j: 34
j: 35
j: 36
j: 37
j: 38
j: 39
j: 40
j: 41
j: 42
j: 43
j: 44
j: 45
j: 46
j: 47
j: 48
j: 49
j: 50
j: 51
j: 52
j: 53
j: 54
j: 55
j: 56
j: 57
j: 58
j: 59
j: 60
j: 61
j: 62
j: 63
j: 64
j: 65
j: 66
j: 67
j: 68
j: 69
j: 70
j: 71
j: 72
j: 73
j: 74
j: 75
j: 76
j: 77
j: 78
j: 79
j: 80
j: 81
j: 82
j: 83
j: 84
j: 85
j: 86
j: 87
j: 88
j: 89
j: 90
j: 91
j: 92
j: 93
j: 94
j: 95
j: 96
j: 97
j: 98
j: 99
j: 100
j: 101
j: 102
j: 103
j: 104
j: 105
j: 106
j: 107
j: 108
j: 109
j: 110
j: 111
j: 112
j: 113
j: 114
j: 115
j: 116
j: 117
j: 118
j: 119
j: 120
j: 121
j: 122
j: 123
j: 124
j: 125
j: 126
j: 127
j: 128
j: 129
j: 130
j: 131
j: 132
j: 133
j: 134
j: 135
j: 136
j: 137
j: 138
j: 139
j: 140
j: 141
j: 142
j: 143
j: 144
j: 145
j: 146
j: 147
j: 148
j: 149
j: 150
j: 151
j: 152
j: 153
j: 154
j: 155
j: 156
j: 157
j: 1

j: 1230
j: 1231
j: 1232
j: 1233
j: 1234
j: 1235
j: 1236
j: 1237
j: 1238
j: 1239
j: 1240
j: 1241
j: 1242
j: 1243
j: 1244
j: 1245
j: 1246
j: 1247
j: 1248
j: 1249
j: 1250
j: 1251
j: 1252
j: 1253
j: 1254
j: 1255
j: 1256
j: 1257
j: 1258
j: 1259
j: 1260
j: 1261
j: 1262
j: 1263
j: 1264
j: 1265
j: 1266
j: 1267
j: 1268
j: 1269
j: 1270
j: 1271
j: 1272
j: 1273
j: 1274
j: 1275
j: 1276
j: 1277
j: 1278
j: 1279
j: 1280
j: 1281
j: 1282
j: 1283
j: 1284
j: 1285
j: 1286
j: 1287
j: 1288
j: 1289
j: 1290
j: 1291
j: 1292
j: 1293
j: 1294
j: 1295
j: 1296
j: 1297
j: 1298
j: 1299
j: 1300
j: 1301
j: 1302
j: 1303
j: 1304
j: 1305
j: 1306
j: 1307
j: 1308
j: 1309
j: 1310
j: 1311
j: 1312
j: 1313
j: 1314
j: 1315
j: 1316
j: 1317
j: 1318
j: 1319
j: 1320
j: 1321
j: 1322
j: 1323
j: 1324
j: 1325
j: 1326
j: 1327
j: 1328
j: 1329
j: 1330
j: 1331
j: 1332
j: 1333
j: 1334
j: 1335
j: 1336
j: 1337
j: 1338
j: 1339
j: 1340
j: 1341
j: 1342
j: 1343
j: 1344
j: 1345
j: 1346
j: 1347
j: 1348
j: 1349
j: 1350
j: 1351
j: 1352
j: 1353
j: 1354


j: 2357
j: 2358
j: 2359
j: 2360
j: 2361
j: 2362
j: 2363
j: 2364
j: 2365
j: 2366
j: 2367
j: 2368
j: 2369
j: 2370
j: 2371
j: 2372
j: 2373
j: 2374
j: 2375
j: 2376
j: 2377
j: 2378
j: 2379
j: 2380
j: 2381
j: 2382
j: 2383
j: 2384
j: 2385
j: 2386
j: 2387
j: 2388
j: 2389
j: 2390
j: 2391
j: 2392
j: 2393
j: 2394
j: 2395
j: 2396
j: 2397
j: 2398
j: 2399
j: 2400
j: 2401
j: 2402
j: 2403
j: 2404
j: 2405
j: 2406
j: 2407
j: 2408
j: 2409
j: 2410
j: 2411
j: 2412
j: 2413
j: 2414
j: 2415
j: 2416
j: 2417
j: 2418
j: 2419
j: 2420
j: 2421
j: 2422
j: 2423
j: 2424
j: 2425
j: 2426
j: 2427
j: 2428
j: 2429
j: 2430
j: 2431
j: 2432
j: 2433
j: 2434
j: 2435
j: 2436
j: 2437
j: 2438
j: 2439
j: 2440
j: 2441
j: 2442
j: 2443
j: 2444
j: 2445
j: 2446
j: 2447
j: 2448
j: 2449
j: 2450
j: 2451
j: 2452
j: 2453
j: 2454
j: 2455
j: 2456
j: 2457
j: 2458
j: 2459
j: 2460
j: 2461
j: 2462
j: 2463
j: 2464
j: 2465
j: 2466
j: 2467
j: 2468
j: 2469
j: 2470
j: 2471
j: 2472
j: 2473
j: 2474
j: 2475
j: 2476
j: 2477
j: 2478
j: 2479
j: 2480
j: 2481


j: 3471
j: 3472
j: 3473
j: 3474
j: 3475
j: 3476
j: 3477
j: 3478
j: 3479
j: 3480
j: 3481
j: 3482
j: 3483
j: 3484
j: 3485
j: 3486
j: 3487
j: 3488
j: 3489
j: 3490
j: 3491
j: 3492
j: 3493
j: 3494
j: 3495
j: 3496
j: 3497
j: 3498
j: 3499
j: 3500
j: 3501
j: 3502
j: 3503
j: 3504
j: 3505
j: 3506
j: 3507
j: 3508
j: 3509
j: 3510
j: 3511
j: 3512
j: 3513
j: 3514
j: 3515
j: 3516
j: 3517
j: 3518
j: 3519
j: 3520
j: 3521
j: 3522
j: 3523
j: 3524
j: 3525
j: 3526
j: 3527
j: 3528
j: 3529
j: 3530
j: 3531
j: 3532
j: 3533
j: 3534
j: 3535
j: 3536
j: 3537
j: 3538
j: 3539
j: 3540
j: 3541
j: 3542
j: 3543
j: 3544
j: 3545
j: 3546
j: 3547
j: 3548
j: 3549
j: 3550
j: 3551
j: 3552
j: 3553
j: 3554
j: 3555
j: 3556
j: 3557
j: 3558
j: 3559
j: 3560
j: 3561
j: 3562
j: 3563
j: 3564
j: 3565
j: 3566
j: 3567
j: 3568
j: 3569
j: 3570
j: 3571
j: 3572
j: 3573
j: 3574
j: 3575
j: 3576
j: 3577
j: 3578
j: 3579
j: 3580
j: 3581
j: 3582
j: 3583
j: 3584
j: 3585
j: 3586
j: 3587
j: 3588
j: 3589
j: 3590
j: 3591
j: 3592
j: 3593
j: 3594
j: 3595


j: 4725
j: 4726
j: 4727
j: 4728
j: 4729
j: 4730
j: 4731
j: 4732
j: 4733
j: 4734
j: 4735
j: 4736
j: 4737
j: 4738
j: 4739
j: 4740
j: 4741
j: 4742
j: 4743
j: 4744
j: 4745
j: 4746
j: 4747
j: 4748
j: 4749
j: 4750
j: 4751
j: 4752
j: 4753
j: 4754
j: 4755
j: 4756
j: 4757
j: 4758
j: 4759
j: 4760
j: 4761
j: 4762
j: 4763
j: 4764
j: 4765
j: 4766
j: 4767
j: 4768
j: 4769
j: 4770
j: 4771
j: 4772
j: 4773
j: 4774
j: 4775
j: 4776
j: 4777
j: 4778
j: 4779
j: 4780
j: 4781
j: 4782
j: 4783
j: 4784
j: 4785
j: 4786
j: 4787
j: 4788
j: 4789
j: 4790
j: 4791
j: 4792
j: 4793
j: 4794
j: 4795
j: 4796
j: 4797
j: 4798
j: 4799
j: 4800
j: 4801
j: 4802
j: 4803
j: 4804
j: 4805
j: 4806
j: 4807
j: 4808
j: 4809
j: 4810
j: 4811
j: 4812
j: 4813
j: 4814
j: 4815
j: 4816
j: 4817
j: 4818
j: 4819
j: 4820
j: 4821
j: 4822
j: 4823
j: 4824
j: 4825
j: 4826
j: 4827
j: 4828
j: 4829
j: 4830
j: 4831
j: 4832
j: 4833
j: 4834
j: 4835
j: 4836
j: 4837
j: 4838
j: 4839
j: 4840
j: 4841
j: 4842
j: 4843
j: 4844
j: 4845
j: 4846
j: 4847
j: 4848
j: 4849


...

j: 84095
j: 84096
j: 84097
j: 84098
j: 84099
j: 84100
j: 84101
j: 84102
j: 84103
j: 84104
j: 84105
j: 84106
j: 84107
j: 84108
j: 84109
j: 84110
j: 84111
j: 84112
j: 84113
j: 84114
j: 84115
j: 84116
j: 84117
j: 84118
j: 84119
j: 84120
j: 84121
j: 84122
j: 84123
j: 84124
j: 84125
j: 84126
j: 84127
j: 84128
j: 84129
j: 84130
j: 84131
j: 84132
j: 84133
j: 84134
j: 84135
j: 84136
j: 84137
j: 84138
j: 84139
j: 84140
j: 84141
j: 84142
j: 84143
j: 84144
j: 84145
j: 84146
j: 84147
j: 84148
j: 84149
j: 84150
j: 84151
j: 84152
j: 84153
j: 84154
j: 84155
j: 84156
j: 84157
j: 84158
j: 84159
j: 84160
j: 84161
j: 84162
j: 84163
j: 84164
j: 84165
j: 84166
j: 84167
j: 84168
j: 84169
j: 84170
j: 84171
j: 84172
j: 84173
j: 84174
j: 84175
j: 84176
j: 84177
j: 84178
j: 84179
j: 84180
j: 84181
j: 84182
j: 84183
j: 84184
j: 84185
j: 84186
j: 84187
j: 84188
j: 84189
j: 84190
j: 84191
j: 84192
j: 84193
j: 84194
j: 84195
j: 84196
j: 84197
j: 84198
j: 84199
j: 84200
j: 84201
j: 84202
j: 84203
j: 84204
j: 84205
j

j: 85013
j: 85014
j: 85015
j: 85016
j: 85017
j: 85018
j: 85019
j: 85020
j: 85021
j: 85022
j: 85023
j: 85024
j: 85025
j: 85026
j: 85027
j: 85028
j: 85029
j: 85030
j: 85031
j: 85032
j: 85033
j: 85034
j: 85035
j: 85036
j: 85037
j: 85038
j: 85039
j: 85040
j: 85041
j: 85042
j: 85043
j: 85044
j: 85045
j: 85046
j: 85047
j: 85048
j: 85049
j: 85050
j: 85051
j: 85052
j: 85053
j: 85054
j: 85055
j: 85056
j: 85057
j: 85058
j: 85059
j: 85060
j: 85061
j: 85062
j: 85063
j: 85064
j: 85065
j: 85066
j: 85067
j: 85068
j: 85069
j: 85070
j: 85071
j: 85072
j: 85073
j: 85074
j: 85075
j: 85076
j: 85077
j: 85078
j: 85079
j: 85080
j: 85081
j: 85082
j: 85083
j: 85084
j: 85085
j: 85086
j: 85087
j: 85088
j: 85089
j: 85090
j: 85091
j: 85092
j: 85093
j: 85094
j: 85095
j: 85096
j: 85097
j: 85098
j: 85099
j: 85100
j: 85101
j: 85102
j: 85103
j: 85104
j: 85105
j: 85106
j: 85107
j: 85108
j: 85109
j: 85110
j: 85111
j: 85112
j: 85113
j: 85114
j: 85115
j: 85116
j: 85117
j: 85118
j: 85119
j: 85120
j: 85121
j: 85122
j: 85123
j

j: 85927
j: 85928
j: 85929
j: 85930
j: 85931
j: 85932
j: 85933
j: 85934
j: 85935
j: 85936
j: 85937
j: 85938
j: 85939
j: 85940
j: 85941
j: 85942
j: 85943
j: 85944
j: 85945
j: 85946
j: 85947
j: 85948
j: 85949
j: 85950
j: 85951
j: 85952
j: 85953
j: 85954
j: 85955
j: 85956
j: 85957
j: 85958
j: 85959
j: 85960
j: 85961
j: 85962
j: 85963
j: 85964
j: 85965
j: 85966
j: 85967
j: 85968
j: 85969
j: 85970
j: 85971
j: 85972
j: 85973
j: 85974
j: 85975
j: 85976
j: 85977
j: 85978
j: 85979
j: 85980
j: 85981
j: 85982
j: 85983
j: 85984
j: 85985
j: 85986
j: 85987
j: 85988
j: 85989
j: 85990
j: 85991
j: 85992
j: 85993
j: 85994
j: 85995
j: 85996
j: 85997
j: 85998
j: 85999
j: 86000
j: 86001
j: 86002
j: 86003
j: 86004
j: 86005
j: 86006
j: 86007
j: 86008
j: 86009
j: 86010
j: 86011
j: 86012
j: 86013
j: 86014
j: 86015
j: 86016
j: 86017
j: 86018
j: 86019
j: 86020
j: 86021
j: 86022
j: 86023
j: 86024
j: 86025
j: 86026
j: 86027
j: 86028
j: 86029
j: 86030
j: 86031
j: 86032
j: 86033
j: 86034
j: 86035
j: 86036
j: 86037
j

j: 86858
j: 86859
j: 86860
j: 86861
j: 86862
j: 86863
j: 86864
j: 86865
j: 86866
j: 86867
j: 86868
j: 86869
j: 86870
j: 86871
j: 86872
j: 86873
j: 86874
j: 86875
j: 86876
j: 86877
j: 86878
j: 86879
j: 86880
j: 86881
j: 86882
j: 86883
j: 86884
j: 86885
j: 86886
j: 86887
j: 86888
j: 86889
j: 86890
j: 86891
j: 86892
j: 86893
j: 86894
j: 86895
j: 86896
j: 86897
j: 86898
j: 86899
j: 86900
j: 86901
j: 86902
j: 86903
j: 86904
j: 86905
j: 86906
j: 86907
j: 86908
j: 86909
j: 86910
j: 86911
j: 86912
j: 86913
j: 86914
j: 86915
j: 86916
j: 86917
j: 86918
j: 86919
j: 86920
j: 86921
j: 86922
j: 86923
j: 86924
j: 86925
j: 86926
j: 86927
j: 86928
j: 86929
j: 86930
j: 86931
j: 86932
j: 86933
j: 86934
j: 86935
j: 86936
j: 86937
j: 86938
j: 86939
j: 86940
j: 86941
j: 86942
j: 86943
j: 86944
j: 86945
j: 86946
j: 86947
j: 86948
j: 86949
j: 86950
j: 86951
j: 86952
j: 86953
j: 86954
j: 86955
j: 86956
j: 86957
j: 86958
j: 86959
j: 86960
j: 86961
j: 86962
j: 86963
j: 86964
j: 86965
j: 86966
j: 86967
j: 86968
j

j: 87773
j: 87774
j: 87775
j: 87776
j: 87777
j: 87778
j: 87779
j: 87780
j: 87781
j: 87782
j: 87783
j: 87784
j: 87785
j: 87786
j: 87787
j: 87788
j: 87789
j: 87790
j: 87791
j: 87792
j: 87793
j: 87794
j: 87795
j: 87796
j: 87797
j: 87798
j: 87799
j: 87800
j: 87801
j: 87802
j: 87803
j: 87804
j: 87805
j: 87806
j: 87807
j: 87808
j: 87809
j: 87810
j: 87811
j: 87812
j: 87813
j: 87814
j: 87815
j: 87816
j: 87817
j: 87818
j: 87819
j: 87820
j: 87821
j: 87822
j: 87823
j: 87824
j: 87825
j: 87826
j: 87827
j: 87828
j: 87829
j: 87830
j: 87831
j: 87832
j: 87833
j: 87834
j: 87835
j: 87836
j: 87837
j: 87838
j: 87839
j: 87840
j: 87841
j: 87842
j: 87843
j: 87844
j: 87845
j: 87846
j: 87847
j: 87848
j: 87849
j: 87850
j: 87851
j: 87852
j: 87853
j: 87854
j: 87855
j: 87856
j: 87857
j: 87858
j: 87859
j: 87860
j: 87861
j: 87862
j: 87863
j: 87864
j: 87865
j: 87866
j: 87867
j: 87868
j: 87869
j: 87870
j: 87871
j: 87872
j: 87873
j: 87874
j: 87875
j: 87876
j: 87877
j: 87878
j: 87879
j: 87880
j: 87881
j: 87882
j: 87883
j

In [4]:
users = df['User'].unique()
users = list(users)
# users.pop(0)
users

[nan,
 'User_111',
 'User_113',
 'User_137',
 'User_45',
 'User_68',
 'User_44',
 'User_66',
 'User_117',
 'User_49',
 'User_105',
 'User_127',
 'User_98',
 'User_39',
 'User_35',
 'User_108',
 'User_72',
 'User_135',
 'User_82',
 'User_74',
 'User_88',
 'User_37',
 'User_7',
 'User_138',
 'User_50',
 'User_10',
 'User_109',
 'User_131',
 'User_133',
 'User_70',
 'User_83',
 'User_43',
 'User_110',
 'User_60',
 'User_9',
 'User_75',
 'User_11',
 'User_96',
 'User_107',
 'User_62',
 'User_55',
 'User_100',
 'User_141',
 'User_130',
 'User_119',
 'User_78',
 'User_17',
 'User_32',
 'User_115',
 'User_122',
 'User_95',
 'User_140',
 'User_102',
 'User_121',
 'User_99',
 'User_132',
 'User_36',
 'User_80',
 'User_129',
 'User_42',
 'User_15',
 'User_73',
 'User_123',
 'User_57',
 'User_61',
 'User_136',
 'User_97',
 'User_22',
 'User_58',
 'User_124',
 'User_51',
 'User_38',
 'User_26',
 'User_81',
 'User_69',
 'User_20',
 'User_19',
 'User_18',
 'User_54',
 'User_85',
 'User_134',
 'User_

In [ ]:
# возможно надо два раза пробежаться по массиву первый раз надо запонить какой пользователь какое событие выполнял
# и в каком кейсе
# во втрой раз роверить на нарушения

In [3]:
users = df['PO TYPE'].unique()
users

array(['Заказ на оформление закупки', 'Контракт'], dtype=object)